In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy
import matplotlib.pyplot as plt

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
])

In [3]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
batch_size = 32
num_epoch = 10

In [5]:
train_data = torchvision.datasets.MNIST(root='/content/VGG_Net', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)

test_data = torchvision.datasets.MNIST(root='/content/VGG_Net', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [6]:
from torch.nn.modules.pooling import AdaptiveMaxPool2d
class VGG(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(1,64,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(64,64,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(64,128,3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(128,256,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(256,256,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(256,256,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(256,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(512,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(512,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(512,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(512,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.Conv2d(512,512,3,stride=1, padding=1),
        nn.ReLU(True),
        nn.MaxPool2d(2,2),
    )

    self.classifiers = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(True),
        nn.Dropout(0.5),
        nn.Linear(4096,4096),
        nn.ReLU(True),
        nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )

  def forward(self, x):
    x = self.conv(x)
    x = x.view(-1, 512*7*7)
    x = self.classifiers(x)
    x = nn.Softmax(1)(x)
    return x


In [7]:
model = VGG()
model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.7)

In [ ]:
for epoch in range(num_epoch):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    predicted = model(images)
    loss = loss_func(predicted, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%1000 == 0:
      print(f"[{epoch}, {i+1}], loss : {loss}")


In [ ]:
correct = total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output, dim=1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

  print(f"Accuracy of the Model : {100*(correct/total)}%")